In [ ]:
import nibabel as nib
import numpy as np
import os
from scipy.ndimage import zoom
from pathlib import Path

# === CONFIG ===
input_root = Path("D:/Dissertation/NIfTI_Output")
output_root = Path("D:/Dissertation/Preprocessed")
target_shape = (128, 128, 128)

# === FUNCTIONS ===
def normalize(volume):
    volume = volume.astype(np.float32)
    volume = (volume - np.mean(volume)) / np.std(volume)
    return volume

def resize_volume(volume, target_shape):
    current_shape = volume.shape
    zoom_factors = [t / c for t, c in zip(target_shape, current_shape)]
    return zoom(volume, zoom_factors, order=1)  # order=1 = linear interpolation

def preprocess_and_save(nii_path, output_path):
    img = nib.load(str(nii_path))
    volume = img.get_fdata()
    volume = normalize(volume)
    volume = resize_volume(volume, target_shape)

    new_img = nib.Nifti1Image(volume, np.eye(4))
    nib.save(new_img, str(output_path))

# === PROCESSING LOOP ===
for label in ["CN", "MCI", "AD"]:
    input_folder = input_root / label
    output_folder = output_root / label
    output_folder.mkdir(parents=True, exist_ok=True)

    for nii_file in input_folder.glob("*.nii.gz"):
        subject_id = nii_file.stem
        output_path = output_folder / f"{subject_id}.nii.gz"

        try:
            preprocess_and_save(nii_file, output_path)
            print(f"✅ Processed: {subject_id}")
        except Exception as e:
            print(f"❌ Error processing {subject_id}: {e}")


In [ ]:
# ==================
# Skull-stripping
# ==================
!pip -q install nibabel tqdm
try:
    import HD_BET
except Exception:
    !pip -q install hd-bet

import subprocess
from pathlib import Path
from tqdm import tqdm
import torch
import re

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

# ---- Paths ----
SRC_DIR = Path("/content/drive/MyDrive/OASIS_New/MCI")  # source
DST_DIR = Path("/content/drive/MyDrive/OASIS3_Preprocessed_Data/MCI_New")  # destination (only *_SS.nii.gz brains)
DST_DIR.mkdir(parents=True, exist_ok=True)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("HD-BET device:", DEVICE)

def is_nii(p: Path) -> bool:
    n = p.name.lower()
    return n.endswith(".nii") or n.endswith(".nii.gz")

def ensure_ss_name(src_name: str) -> str:
    """Make a single output name with _SS and .nii.gz extension."""
    if src_name.lower().endswith(".nii.gz"):
        stem, ext = src_name[:-7], ".nii.gz"
    elif src_name.lower().endswith(".nii"):
        stem, ext = src_name[:-4], ".nii.gz"
    else:
        stem, ext = Path(src_name).stem, ".nii.gz"
    if not stem.endswith("_SS"):
        stem += "_SS"
    return f"{stem}{ext}"

def run_hdbet(in_file: Path, out_file: Path) -> bool:
    """
    Call hd-bet to create ONE output: the skull-stripped image at out_file.
    We DO NOT request the mask to avoid *_SS_bet duplicates.
    """
    cmd = [
        "hd-bet",
        "-i", str(in_file),
        "-o", str(out_file),
        "-device", DEVICE,
        # DO NOT USE: "--save_bet_mask"

    ]
    res = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if res.returncode != 0:
        last = (res.stderr or res.stdout).splitlines()[-1] if (res.stderr or res.stdout) else "unknown error"
        print(f"  ❌ HD-BET failed: {in_file.name} -> {last}")
        return False
    return True

# ---------- CLEANUP HELPERS ----------
MASK_SUFFIX_PATTERNS = [
    re.compile(r"_SS_bet\.nii\.gz$", re.IGNORECASE),
    re.compile(r"_bet\.nii\.gz$", re.IGNORECASE),
    re.compile(r"_mask\.nii\.gz$", re.IGNORECASE),
]

def looks_like_mask(name: str) -> bool:
    lower = name.lower()
    return any(p.search(lower) for p in MASK_SUFFIX_PATTERNS)

def cleanup_masks(dst_dir: Path):
    """
    Remove any lingering mask files like *_SS_bet.nii.gz, *_bet.nii.gz, *_mask.nii.gz
    IF the paired *_SS.nii.gz brain image exists. Otherwise leave them alone.
    """
    deleted = 0
    for p in sorted(dst_dir.glob("*.nii*")):
        if not p.is_file():
            continue
        if looks_like_mask(p.name):
            base = p.name
            base = re.sub(r"_SS_bet\.nii\.gz$", "_SS.nii.gz", base, flags=re.IGNORECASE)
            base = re.sub(r"_bet\.nii\.gz$", "_SS.nii.gz",   base, flags=re.IGNORECASE)
            base = re.sub(r"_mask\.nii\.gz$", "_SS.nii.gz",  base, flags=re.IGNORECASE)
            paired = dst_dir / base
            if paired.exists():
                try:
                    p.unlink()
                    deleted += 1
                except Exception as e:
                    print(f"  ⚠️ Could not delete {p.name}: {e}")
    return deleted

# ---------- MAIN ----------
# 1) Clean any old mask files (if a good *_SS exists)
deleted_masks = cleanup_masks(DST_DIR)
if deleted_masks:
    print(f"🧹 Removed {deleted_masks} stray mask files from {DST_DIR}.")

# 2) Process sources
src_files = [p for p in sorted(SRC_DIR.rglob("*")) if p.is_file() and is_nii(p)]
print(f"Found {len(src_files)} AD files to check/convert")

converted, skipped, failed = 0, 0, []

for p in tqdm(src_files, desc="Skull stripping AD", unit="file"):
    out_name = ensure_ss_name(p.name)
    out_path = DST_DIR / out_name

    # If the desired brain-extracted file already exists, just skip
    if out_path.exists():
        skipped += 1
        continue

    # Otherwise run hd-bet to create the _SS file ONLY
    if run_hdbet(p, out_path):
        converted += 1
    else:
        failed.append(str(p))

# 3) Final cleanup pass (just in case)
deleted_masks_2 = cleanup_masks(DST_DIR)
deleted_masks += deleted_masks_2
if deleted_masks_2:
    print(f"🧹 Removed {deleted_masks_2} more stray mask files.")

print("\n=== SUMMARY (AD) ===")
print(f"Total in source     : {len(src_files)}")
print(f"Converted new files : {converted}")
print(f"Skipped (already _SS): {skipped}")
print(f"Deleted mask files  : {deleted_masks}")
print(f"Failed conversions  : {len(failed)}")
if failed:
    print("First few failures:")
    for f in failed[:10]:
        print("  -", f)
print("Output folder:", DST_DIR)


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 113.0 MB/s eta 0:0

Skull stripping AD: 100%|██████████| 297/297 [4:16:28<00:00, 51.81s/file]


=== SUMMARY (AD) ===
Total in source     : 297
Converted new files : 297
Skipped (already _SS): 0
Deleted mask files  : 0
Failed conversions  : 0
Output folder: /content/drive/MyDrive/OASIS3_Preprocessed_Data/MCI_New
